In [1]:
import os
import doctest
from collections import OrderedDict
from llparser import *
from tokens import *
! python tokens.py

Traceback (most recent call last):
  File "/home/feelus/tmp/mycpp/tokens.py", line 656, in <module>
    read('/** @file form3.h',mkpos(0),sequence(a=spcs, b=rep_star(token_s)))
                                                                  ^^^^^^^
NameError: name 'token_s' is not defined


In [2]:
os.chdir('../form/sources')
os.getcwd()

'/home/feelus/tmp/form/sources'

In [3]:
with open('form3.h', 'r') as file1:
	Lines = [x for x in enumerate(file1.read().split(sep='\n'),1)]
Lines

[(1, '/** @file form3.h'),
 (2, ' *'),
 (3, ' *  Contains critical defines for the compilation process'),
 (4, ' *\tAlso contains the inclusion of all necessary header files.'),
 (5, ' *\tThere are also some system dependencies concerning file functions.'),
 (6, ' */'),
 (7, ''),
 (8, '/* #[ License : */'),
 (9, '/*'),
 (10, ' *   Copyright (C) 1984-2023 J.A.M. Vermaseren'),
 (11,
  ' *   When using this file you are requested to refer to the publication'),
 (12, ' *   J.A.M.Vermaseren "New features of FORM" math-ph/0010025'),
 (13,
  ' *   This is considered a matter of courtesy as the development was paid'),
 (14,
  ' *   for by FOM the Dutch physics granting agency and we would like to'),
 (15, ' *   be able to track its scientific use to convince FOM of its value'),
 (16, ' *   for the community.'),
 (17, ' *'),
 (18, ' *   This file is part of FORM.'),
 (19, ' *'),
 (20,
  ' *   FORM is free software: you can redistribute it and/or modify it under the'),
 (21,
  ' *   terms of the

In [4]:
'a\n'.split(sep='\n')

['a', '']

In [5]:
def printpos(*l):
	## todo чтобы можно было использовать с непечатаемыми символами
	l = sorted(l)
	ll = []
	minus = []
	for x in l:
		if x<0:
			minus.append(x)
			continue
		if len(ll)==0:
			ll.append((x,1))
		else:
			if x==ll[-1][0]:
				ll[-1] = (x,ll[-1][1]+1)
			else:
				ll.append((x,1))
			
	cursors = '^%$#@'
	p=0
	for x,n in ll:
		assert x-p>=0
		print(' '*(x-p)+cursors[n-1],end='')
		p=x+1
	print(' |',*minus)
print('aaabbbccddd')
printpos(3,6,8,11)
printpos(3,3,5,4,3,-1,-5)
printpos()

aaabbbccddd
   ^  ^ ^  ^ |
   $^^ | -5 -1
 |


In [6]:
print('---')
print('\n')
print('---')


---


---


In [7]:
def strip_baks_comment(lines):
	r'''
	>>> [x for x in strip_baks_comment([(1,'aaa /*$bbbb$*/cccc'), (2,'aaa //$bbb\\'), (3,'aaa /*$bbbb$*/cccc /*$zzz$*/aaa //$bbb'), (4,'abc')])]
	[(1, 'aaacccc'), (2, 'aaa\\'), (3, 'aaaccccaaa'), (4, 'abc')]
	'''
	for ln,l in lines:
		#print()
		#print(l+'|')
		r = ''
		p=0
		while (q:=l[p:].find(' /*$')) !=-1:
			q+=p
			r+=l[p:q]
			p = l[q+3:].index('$*/')+3+q+3
			#printpos(q,p)
		if (q:= l[p:].find(' //$')) != -1:
			q+=p
			r+=l[p:q]
			if l[-1]=='\\':
				r+='\\'
			#printpos(q)
		else:
			r+=l[p:]
		yield (ln,r)


In [8]:
[x for x in strip_baks_comment(Lines)]
	

[(1, '/** @file form3.h'),
 (2, ' *'),
 (3, ' *  Contains critical defines for the compilation process'),
 (4, ' *\tAlso contains the inclusion of all necessary header files.'),
 (5, ' *\tThere are also some system dependencies concerning file functions.'),
 (6, ' */'),
 (7, ''),
 (8, '/* #[ License : */'),
 (9, '/*'),
 (10, ' *   Copyright (C) 1984-2023 J.A.M. Vermaseren'),
 (11,
  ' *   When using this file you are requested to refer to the publication'),
 (12, ' *   J.A.M.Vermaseren "New features of FORM" math-ph/0010025'),
 (13,
  ' *   This is considered a matter of courtesy as the development was paid'),
 (14,
  ' *   for by FOM the Dutch physics granting agency and we would like to'),
 (15, ' *   be able to track its scientific use to convince FOM of its value'),
 (16, ' *   for the community.'),
 (17, ' *'),
 (18, ' *   This file is part of FORM.'),
 (19, ' *'),
 (20,
  ' *   FORM is free software: you can redistribute it and/or modify it under the'),
 (21,
  ' *   terms of the

In [9]:
def end_merge(lines):
	q=r'''
	>>> [x for x in end_merge([777,'asd'])]
	[(777, 'asd', [])]
	'''
	r=''
	sep_positions = []
	for ln,l in lines:
		if not l.endswith('\\'):
			yield (ln,r+l,sep_positions)
			r=''
			sep_positions=[]
		else:
			r+=l.removesuffix('\\')
			sep_positions.append(sep_positions[-1]+len(l)-1 if len(sep_positions) else len(l)-1)
	if len(sep_positions):
		raise ValueError('aaa')
doctest.testmod()

TestResults(failed=0, attempted=1)

In [10]:
for ln,x,sp in end_merge(strip_baks_comment(Lines)):
    if len(sp):
        print(ln,sp,repr(x))

185 [30] '#define STATIC_ASSERT__3(X,L) \ttypedef char static_assertion_failed_##L[(!!(X))*2-1]'
402 [29] '#define PADDUMMY(type, size) \tUBYTE d_u_m_m_y[alignof(type) - ((size) & (alignof(type) - 1))]'
410 [49, 66, 93, 121, 148, 176, 204] '#define PADPOSITION(ptr_,long_,int_,word_,byte_) \tPADDUMMY(off_t, \t\t+ sizeof(int *) * (ptr_) \t\t+ sizeof(LONG)  * (long_) \t\t+ sizeof(int)   * (int_) \t\t+ sizeof(WORD)  * (word_) \t\t+ sizeof(UBYTE) * (byte_) \t)'
417 [43, 60, 88, 115, 143, 171] '#define PADPOINTER(long_,int_,word_,byte_) \tPADDUMMY(int *, \t\t+ sizeof(LONG)  * (long_) \t\t+ sizeof(int)   * (int_) \t\t+ sizeof(WORD)  * (word_) \t\t+ sizeof(UBYTE) * (byte_) \t)'
423 [34, 50, 77, 105, 133] '#define PADLONG(int_,word_,byte_) \tPADDUMMY(LONG, \t\t+ sizeof(int)   * (int_) \t\t+ sizeof(WORD)  * (word_) \t\t+ sizeof(UBYTE) * (byte_) \t)'
428 [28, 43, 71, 99] '#define PADINT(word_,byte_) \tPADDUMMY(int, \t\t+ sizeof(WORD)  * (word_) \t\t+ sizeof(UBYTE) * (byte_) \t)'
432 [23, 39, 67] 

In [11]:
@cacheread
def token_s(s,p):
	return read_atleast_oneof(s,p,
		id=sequence(a=identifier        ,b=spcs),
		p=sequence(a=punctuator        ,b=spcs),
		int=sequence(a=integer_constant  ,b=spcs),
		fl=sequence(a=floating_constant ,b=spcs),
		ch=sequence(a=character_constant,b=spcs),
		str=sequence(a=string_literal    ,b=spcs),
		proc=select_longest,
		errproc='token_s'
	)

In [12]:
def highlight_comments(lines):
	r'''
	/*x
	x //$ comment
	x //$ comment\
	x*/
	
	//x\
	x /*$ comment $*/\
	x /*$ comment $*/
	'''
	def read_rest_multiline(has_first):
		nonlocal prev_fined
		# clozure : s, pos, sp, ol_insertions
		start = pos.x
		zzz = read(s,pos,rest_multiline_comment)
		assert isok(zzz)
		if not zzz.finalized: # ... \n
			for x in sp:
				if x>=pos.x:
					if has_first:
						ol_insertions.append(x)
					else:
						has_first = True
			if has_first:
				ol_insertions.append(len(s))
			prev_fined = False
		else:
			end = pos.x #  ... */
			for x in sp:
				if x>=start and x<end:
					if has_first:
						ol_insertions.append(x)
					else:
						has_first = True
			prev_fined = True

	ml_insertions = []
	ol_insertions = []
	prev_fined = True # finalized
	
	for ln,s,sp in lines:
		pos = mkpos(0)
		if not prev_fined: # ... \n or ... */
			read_rest_multiline(True)
		if prev_fined:
			last = pos.x-1
			while pos.x!=len(s):
				if pos.x==last:
					print(len(s),pos.x,repr(s))
					raise Exception()
				else:
					last = pos.x
				try:
					read(s,pos,sequence(a=spcs, b=rep_star(token_s)))
				except BaseException as e:
					print(ln,repr(s))
					raise e
				comm = read(s,pos,start_oneline_comment)
				if isok(comm): # // .... \n
					has_first = False
					for x in sp:
						if x>=pos.x:
							if has_first:
								ml_insertions.append(x)
							else:
								has_first = True
					if has_first:
						ml_insertions.append(len(s))
					read(s,pos,rest_oneline_comment)
				comm = read(s,pos,start_multiline_comment)
				if isok(comm):
					read_rest_multiline(False) # /* ... \n or /* ... */
		assert pos.x==len(s)
		if len(ml_insertions) and len(ol_insertions):
			assert ol_insertions[-1]< ml_insertions[0]
		
		newsp = []
		news = ''
		pos = 0
		cum_mov = 0
		while len(ml_insertions) or len(ol_insertions) or len(sp):
			if len(ol_insertions) and (len(sp)==0 or ol_insertions[0]<=sp[0]):
				news+= s[pos:ol_insertions[0]]
				pos = ol_insertions[0]
				OLC = ' //$ comment'
				news+= OLC
				cum_mov +=len(OLC)
				del ol_insertions[0]
			if len(ol_insertions)==0 and len(ml_insertions) and (len(sp)==0 or ml_insertions[0]<=sp[0]):
				news+= s[pos:ml_insertions[0]]
				pos = ml_insertions[0]
				MLC = ' //$ comment'
				news+= MLC
				cum_mov +=len(MLC)
				del ml_insertions[0]
			if len(sp) and (len(ml_insertions)==0 or sp[0]<ml_insertions[0]) and (len(ol_insertions)==0 or sp[0]<ol_insertions[0]):
				news+= s[pos:sp[0]]
				pos = sp[0]
				newsp.append(sp[0]+cum_mov)
				del sp[0]
		news+= s[pos:]

		yield (ln,news,newsp)

	if not prev_fined:
		print('error in source: multiline comment not finalized')





In [13]:
[x for x in highlight_comments(end_merge(strip_baks_comment(Lines))) ]

[(1, '/** @file form3.h', []),
 (2, ' * //$ comment', []),
 (3,
  ' *  Contains critical defines for the compilation process //$ comment',
  []),
 (4,
  ' *\tAlso contains the inclusion of all necessary header files. //$ comment',
  []),
 (5,
  ' *\tThere are also some system dependencies concerning file functions. //$ comment',
  []),
 (6, ' */', []),
 (7, '', []),
 (8, '/* #[ License : */', []),
 (9, '/*', []),
 (10, ' *   Copyright (C) 1984-2023 J.A.M. Vermaseren //$ comment', []),
 (11,
  ' *   When using this file you are requested to refer to the publication //$ comment',
  []),
 (12,
  ' *   J.A.M.Vermaseren "New features of FORM" math-ph/0010025 //$ comment',
  []),
 (13,
  ' *   This is considered a matter of courtesy as the development was paid //$ comment',
  []),
 (14,
  ' *   for by FOM the Dutch physics granting agency and we would like to //$ comment',
  []),
 (15,
  ' *   be able to track its scientific use to convince FOM of its value //$ comment',
  []),
 (16, ' *   f

In [14]:
def line_split(ln,s,sp):
	cum_mov=0
	news = ''
	pos=0
	while len(sp):
		news+= s[pos:sp[0]]
		pos = sp[0]
		NL = '\\\n'
		news+= NL
		cum_mov +=len(NL)
		del sp[0]
	news+= s[pos:]
	return news
print(line_split(777,'#define Utruncate(f) { \tif ( ftruncate(f->descriptor, 0) ) { \t\tMLOCK(ErrorMessageLock); \t\tMesPrint("Utruncate failed"); \t\tMUNLOCK(ErrorMessageLock); \t\t/* Calling Terminate() here may cause an infinite loop due to CleanUpSort(). */ \t\t/* Terminate(-1); */ \t} }',
  [23, 61, 88, 120, 149, 231, 254, 257]))

#define Utruncate(f) { \
	if ( ftruncate(f->descriptor, 0) ) { \
		MLOCK(ErrorMessageLock); \
		MesPrint("Utruncate failed"); \
		MUNLOCK(ErrorMessageLock); \
		/* Calling Terminate() here may cause an infinite loop due to CleanUpSort(). */ \
		/* Terminate(-1); */ \
	} \
}


In [15]:
with open('form3.h', 'w') as file1:
	#file1.write('\n'.join(line_split(ln,s,sp) for ln,s,sp in highlight_comments(end_merge(strip_baks_comment(Lines)))))
	file1.write('\n'.join(line_split(ln,s,sp) for ln,s,sp in (end_merge(strip_baks_comment(Lines)))))
os.getcwd()

'/home/feelus/tmp/form/sources'

In [16]:
def file_hl_comments(name):
	with open(name, 'r') as file1:
		Lines = [x for x in enumerate(file1.read().split(sep='\n'),1)]
	with open(name, 'w') as file1:
		file1.write('\n'.join(line_split(ln,s,sp) for ln,s,sp in highlight_comments(end_merge(strip_baks_comment(Lines)))))
def file_unhl_comments(name):
	with open(name, 'r') as file1:
		Lines = [x for x in enumerate(file1.read().split(sep='\n'),1)]
	with open(name, 'w') as file1:
		file1.write('\n'.join(line_split(ln,s,sp) for ln,s,sp in (end_merge(strip_baks_comment(Lines)))))

In [17]:
len([ x for x in os.listdir() if not x.endswith('.h')])

63

In [18]:
file_hl_comments('grccparam.h')

In [19]:
read('1.0e-10',mkpos(0),token_s)

mk_fo_dict(a=mk_fo_dict(value=1.0, type='double'), b='')

In [20]:
print('#define GRCC_FRACERROR       (1.0e-10)')
printpos(30)

#define GRCC_FRACERROR       (1.0e-10)
                              ^ |


In [21]:
for i,x in enumerate(os.listdir()):
	if x.endswith('.h') or x.endswith('.c') or x.endswith('.cc'):
		print(i,x)
		file_hl_comments(x)

0 grccparam.h


2 sort.c
3 polygcd.h
4 compiler.c
5 names.c
6 if.c
7 transform.c
8 evaluate.c
9 sch.c
10 comtool.c
11 lus.c
12 gentopo.cc
13 poly.h
14 proces.c
15 token.c
16 normal.c


In [ ]:
def printret(*args):
	print(*[repr(x) for x in args])
	return args[0]


In [ ]:
x=7
try:
	x='123\u0100'.encode(encoding='latin1',errors='surrogatepass')
except Exception as e:
	print(type(e),*[(x,getattr(e,x)) for x in dir(e) if not x.startswith('__')],e,sep='\n')
x

<class 'UnicodeEncodeError'>
('add_note', <built-in method add_note of UnicodeEncodeError object at 0x7f72635c3540>)
('args', ('latin-1', '123Ā', 3, 4, 'ordinal not in range(256)'))
('encoding', 'latin-1')
('end', 4)
('object', '123Ā')
('reason', 'ordinal not in range(256)')
('start', 3)
('with_traceback', <built-in method with_traceback of UnicodeEncodeError object at 0x7f72635c3540>)
'latin-1' codec can't encode character '\u0100' in position 3: ordinal not in range(256)


7

In [ ]:
def gen_fun(name):
	def default_name():
		return 1/0.1
	default_name.__name__ = name
	return default_name
foo = gen_fun('hello')
pass
pass
pass
foo()

10.0